In [1]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from collections import defaultdict
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader, Batch
from torch.utils.data import DataLoader, Subset, Dataset as TorchDataset
from itertools import combinations
import numpy as np
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit
from utils2 import (MultiTaskGAT, EarlyStopping, WeightTracker, split_dataset_by_combinations, 
    load_all_weights, save_checkpoint, train, validation, FocalLoss, load_data, CustomDataset)
import random
import pickle
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)

    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(2)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
rng = np.random.default_rng(8888)
rfloat = rng.random()

In [2]:
task_to_idx = {'liv': 0, 'lun': 1, 'sto': 2, 'mgl': 3}
idx_to_task = {v: k for k, v in task_to_idx.items()}

# Hyperparameters for each task
task_hyperparams = {
'liv': {'lr': 0.00005,  'weight_decay': 1e-03},
'lun': {'lr': 0.001, 'weight_decay': 1e-05},
'sto': {'lr': 0.0001, 'weight_decay': 1e-05},
'mgl': {'lr': 0.0001,  'weight_decay': 1e-05}
}

def load_pretrained_combination_dataloaders(base_dir, task_combinations, batch_size=4, val_ratio=0.2):
    """
    Loads pre-generated data for each combination and returns DataLoaders for training and validation.
    """
    train_combination_dataloaders = {}
    validation_combination_dataloaders = {}

    for combo in task_combinations:
        tissue_comb_name = '_'.join(combo)
        data_path = os.path.join(base_dir, 'val', f"{tissue_comb_name}_data.npy")
        labels_path = os.path.join(base_dir, 'val', f"{tissue_comb_name}_labels.npy")
        smiles_path = os.path.join(base_dir, 'val', f"{tissue_comb_name}_smiles.txt")

        if not (os.path.exists(data_path) and os.path.exists(labels_path) and os.path.exists(smiles_path)):
            print(f"Combination {combo}: Required files do not exist. Skipping.")
            continue

        # Load data
        data_x = np.load(data_path, allow_pickle=True)
        data_y = np.load(labels_path, allow_pickle=True).T  # (num_samples, num_tasks)

        with open(smiles_path, 'r', encoding='utf-8') as f:
            smiles_list = [line.rstrip() for line in f]

        # Create dataset
        dataset = CustomDataset(data_x, data_y, smiles_list)

        # Split into training and validation
        sss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=val_ratio, random_state=1)
        train_idx, val_idx = next(sss.split(np.zeros(len(data_y)), data_y))

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Create DataLoaders
        train_loader = DataLoader(
            train_subset, batch_size=batch_size, shuffle=True, collate_fn=Batch.from_data_list, num_workers=0
        )
        val_loader = DataLoader(
            val_subset, batch_size=batch_size, shuffle=False, collate_fn=Batch.from_data_list, num_workers=0
        )

        # Add to dictionaries
        train_combination_dataloaders[combo] = train_loader
        validation_combination_dataloaders[combo] = val_loader

        print(f"Combination {combo}: Number of training samples = {len(train_subset)}, Number of validation samples = {len(val_subset)}")

    return train_combination_dataloaders, validation_combination_dataloaders

def analyze_gradient_similarity(model, criterion, device, dataloader, selected_tasks, final_layer_ids):
    model.train()  
    batch = next(iter(dataloader))
    batch = batch.to(device)
    
    _, outputs, _ = model(batch, selected_tasks)
    
    shared_params = [p for p in model.parameters() if id(p) not in final_layer_ids]
    
    task_gradients = {}
    for i, global_task in enumerate(selected_tasks):
        model.zero_grad()  
        loss = criterion(outputs[i], batch.y[:, i].float())
        loss.backward(retain_graph=True)

        grads = []
        for p in shared_params:
            if p.grad is not None:
                grads.append(p.grad.detach().clone().flatten())
            else:
                grads.append(torch.zeros_like(p).flatten())
        if grads:
            grad_vector = torch.cat(grads)
        else:
            grad_vector = None
        task_gradients[global_task] = grad_vector
    
    # Calculate cosine similarity for each pair of tasks
    similarities = {}
    tasks = list(task_gradients.keys())
    for i in range(len(tasks)):
        for j in range(i+1, len(tasks)):
            grad_i = task_gradients[tasks[i]]
            grad_j = task_gradients[tasks[j]]
            if grad_i is not None and grad_j is not None:
                cos_sim = F.cosine_similarity(grad_i, grad_j, dim=0).item()
                similarities[(tasks[i], tasks[j])] = cos_sim
    return similarities

def train_model(pretrained_train_dataloaders, pretrained_val_dataloaders, model, optimizer, criterion, device):
    weight_tracker = WeightTracker()
    early_stopping_dict = {}

    task_combinations = list(pretrained_train_dataloaders.keys())
    print(f"\n============================= Starting pre-training with 3-task combinations =============================")

    final_layer_ids = set()
    for layer in model.final_layer:
        for param in layer.parameters():
            final_layer_ids.add(id(param))
    
    last_epoch = None  

    for epoch in range(1, 5000):
        epoch_train_losses = []
        epoch_val_losses = []

        print(f"\n--- Epoch {epoch} ---")
        all_stopped = True

        for combo in task_combinations:
            if combo not in pretrained_train_dataloaders or combo not in pretrained_val_dataloaders:
                continue

            if combo not in early_stopping_dict:
                early_stopping_dict[combo] = EarlyStopping(
                    patience=10, verbose=False,
                    path=f'/data/home/dbswn0814/2025JCM/model/multi task3/model/checkpoint_{combo}.pt'
                )

            if early_stopping_dict[combo].early_stop:
                continue

            all_stopped = False 

            train_loader = pretrained_train_dataloaders[combo]
            val_loader = pretrained_val_dataloaders[combo]

            selected_tasks = tuple([task_to_idx[task] for task in combo])
            train_loss = train(epoch, model, criterion, optimizer, device, train_loader, 
                               selected_tasks=selected_tasks, idx_to_task=idx_to_task)
            val_loss = validation(epoch, model, criterion, device, val_loader, 
                                  selected_tasks=selected_tasks, idx_to_task=idx_to_task)

            aggregate_val_loss = np.mean(val_loss)

            epoch_train_losses.append(train_loss)
            epoch_val_losses.append(val_loss)

            early_stopping_dict[combo](aggregate_val_loss, model, optimizer)
        
        if all_stopped:
            last_epoch = epoch
            print("All combinations have triggered early stopping.")
            break
        else:
            last_epoch = epoch  

    print(f"\n==> Comparing weights after pre-training with 3-task combinations:")
    weight_tracker.compare_weights_after_training(model)

    # Save the first step model
    save_checkpoint(model, optimizer, '/data/home/dbswn0814/2025JCM/model/multi task3/model/checkpoint_layer2_pretrained.pth.tar')
    print(f"\n============================= Pre-training with 3-task combinations completed at epoch {last_epoch} =============================\n")

    print("\n=== Final Gradient Similarity Analysis ===")
    for combo in task_combinations:
        if combo in pretrained_train_dataloaders:
            selected_tasks = tuple([task_to_idx[task] for task in combo])
            gradient_similarities = analyze_gradient_similarity(model, criterion, device, 
                                                                pretrained_train_dataloaders[combo], 
                                                                selected_tasks, final_layer_ids)
            print(f"Final Gradient similarity for combo {combo} (tasks {selected_tasks}): {gradient_similarities}")

def main():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    base_dir = '/data/home/dbswn0814/2025JCM/data/multi task3'
    tissues = ['liv', 'lun', 'sto', 'mgl']
    task_combinations = [combo for combo in combinations(tissues, 3)]

    pretrained_train_dataloaders, pretrained_val_dataloaders = load_pretrained_combination_dataloaders(
        base_dir=base_dir,
        task_combinations=task_combinations,
        batch_size=2,
        val_ratio=0.1
    )

    model = MultiTaskGAT(num_features=75, n_heads1=2, output_dim_idx=16, dropout=0.2).to(device)
    
    task_params_ids = {}
    for task, idx in task_to_idx.items():
        task_params_ids[task] = set(id(p) for p in model.final_layer[idx].parameters())
    
    optimizer = torch.optim.Adam([
        {'params': [p for p in model.parameters() if id(p) in task_params_ids['liv']],
         'lr': task_hyperparams['liv']['lr'],
         'weight_decay': task_hyperparams['liv']['weight_decay']},
        {'params': [p for p in model.parameters() if id(p) in task_params_ids['lun']],
         'lr': task_hyperparams['lun']['lr'],
         'weight_decay': task_hyperparams['lun']['weight_decay']},
        {'params': [p for p in model.parameters() if id(p) in task_params_ids['sto']],
         'lr': task_hyperparams['sto']['lr'],
         'weight_decay': task_hyperparams['sto']['weight_decay']},
        {'params': [p for p in model.parameters() if id(p) in task_params_ids['mgl']],
         'lr': task_hyperparams['mgl']['lr'],
         'weight_decay': task_hyperparams['mgl']['weight_decay']},
        {'params': [p for p in model.parameters() if id(p) not in set.union(*task_params_ids.values())],
         'lr': 0.0001,  
         'weight_decay': 1e-04}  
    ])

    criterion = FocalLoss(alpha=0.25, gamma=4.0, reduction='mean')

    # first step learning
    train_model(pretrained_train_dataloaders, pretrained_val_dataloaders, model, optimizer, criterion, device)

    pretrained_model_path = '/data/home/dbswn0814/2025JCM/model/multi task3/model/layer2_pretrained.pth'
    torch.save(model.state_dict(), pretrained_model_path)
    print(f"Saved pre-trained model weights: {pretrained_model_path}")

    # second step learning(20-fold cross validation)
    dataset = load_data(batch_size=8)
    K = 20  
    kf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=18)
    X = dataset.data_x
    y = dataset.data_y
    data_dir = '/data/home/dbswn0814/2025JCM/model/multi task3/data'
    
    all_fold_gradient_similarities = []

    for fold, (train_indices, test_indices) in enumerate(kf.split(X, y)):
        print(f"\n======================= Fold {fold + 1}/{K} =======================")

        # train, validation 분리
        y_train = y[train_indices]
        sss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=1) #1
        train_idx, val_idx = next(sss.split(np.zeros(len(y_train)), y_train))
        train_indices_new = train_indices[train_idx]
        val_indices = train_indices[val_idx]

        train_dataset = Subset(dataset, train_indices_new)
        validation_dataset = Subset(dataset, val_idx)
        test_dataset = Subset(dataset, test_indices)

        print(f"Fold {fold + 1}: Train={len(train_indices_new)}, Validation={len(val_indices)}, Test={len(test_indices)}")

        # load first step weights
        model = MultiTaskGAT(num_features=75, n_heads1=2, output_dim_idx=16, dropout=0.2).to(device)
        model.load_state_dict(torch.load(pretrained_model_path))
        
        task_params_ids = {}
        for task, idx in task_to_idx.items():
            task_params_ids[task] = set(id(p) for p in model.final_layer[idx].parameters())

        optimizer = torch.optim.Adam([
            {'params': [p for p in model.parameters() if id(p) in task_params_ids['liv']],
             'lr': task_hyperparams['liv']['lr'],
             'weight_decay': task_hyperparams['liv']['weight_decay']},
            {'params': [p for p in model.parameters() if id(p) in task_params_ids['lun']],
             'lr': task_hyperparams['lun']['lr'],
             'weight_decay': task_hyperparams['lun']['weight_decay']},
            {'params': [p for p in model.parameters() if id(p) in task_params_ids['sto']],
             'lr': task_hyperparams['sto']['lr'],
             'weight_decay': task_hyperparams['sto']['weight_decay']},
            {'params': [p for p in model.parameters() if id(p) in task_params_ids['mgl']],
             'lr': task_hyperparams['mgl']['lr'],
             'weight_decay': task_hyperparams['mgl']['weight_decay']},
            {'params': [p for p in model.parameters() if id(p) not in set.union(*task_params_ids.values())],
             'lr': 0.0001, 
             'weight_decay': 1e-04}  
        ])
        
        criterion = FocalLoss(alpha=0.25, gamma=4.0, reduction='mean')

        # Save datasets for fold
        train_save_path = os.path.join(data_dir, f'fold_{fold + 1}_train.pt')
        val_save_path = os.path.join(data_dir, f'fold_{fold + 1}_val.pt')
        test_save_path = os.path.join(data_dir, f'fold_{fold + 1}_test.pt')
        torch.save(train_dataset, train_save_path)
        torch.save(validation_dataset, val_save_path)
        torch.save(test_dataset, test_save_path)
        print(f"Saved datasets for Fold {fold + 1}: {train_save_path}, {val_save_path}, {test_save_path}")

        # DataLoaders 
        full_train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False, 
                                       collate_fn=Batch.from_data_list, num_workers=0)
        full_validation_loader = DataLoader(validation_dataset, batch_size=8, shuffle=False, 
                                            collate_fn=Batch.from_data_list, num_workers=0)

        # EarlyStopping 
        early_stopping = EarlyStopping(patience=10, verbose=False, 
                                       path=f'/data/home/dbswn0814/2025JCM/model/multi task3/model/checkpoint_fold_{fold + 1}.pt', 
                                       trace_func=print)

        print(f"\n============================= Starting retraining with full data =============================")
        for epoch in range(1, 1001):
            train_loss = train(epoch, model, criterion, optimizer, device, train_loader=full_train_loader, 
                                selected_tasks=(0, 1, 2, 3), idx_to_task=idx_to_task)
            val_loss = validation(epoch, model, criterion, device, validation_loader=full_validation_loader, 
                                  selected_tasks=(0, 1, 2, 3), idx_to_task=idx_to_task)
            aggregate_val_loss = np.mean(val_loss)
            early_stopping(aggregate_val_loss, model, optimizer)
            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch}. Best validation loss: {early_stopping.val_loss_min:.4f}")
                break

        # after second step learning, Calculate the final gradient similarity 
        final_layer_ids_fold = set()
        for layer in model.final_layer:
            for param in layer.parameters():
                final_layer_ids_fold.add(id(param))
        print("\n=== Final Gradient Similarity Analysis for Fold {} ===".format(fold+1))
        final_gradient_sim = analyze_gradient_similarity(model, criterion, device, 
                                                         full_train_loader, (0, 1, 2, 3), final_layer_ids_fold)
        print(f"Final Gradient similarity for fold {fold+1} (tasks (0, 1, 2, 3)): {final_gradient_sim}")
        all_fold_gradient_similarities.append(final_gradient_sim)

        save_checkpoint(model, optimizer, f'/data/home/dbswn0814/2025JCM/model/multi task3/model/checkpoint_full_retrained_fold_{fold + 1}.pth.tar')
        print(f"\n============================= Retraining with full data completed for Fold {fold + 1} =============================\n")

        checkpoint = torch.load(f'/data/home/dbswn0814/2025JCM/model/multi task3/model/checkpoint_fold_{fold + 1}.pt')
        model.load_state_dict(checkpoint['state_dict'])
        final_model_path = f'/data/home/dbswn0814/2025JCM/model/multi task3/model/fold_{fold + 1}_final.pth'
        torch.save(model.state_dict(), final_model_path)
        print(f"Saved final model weights for Fold {fold + 1}: {final_model_path}")

    # Calculate the final gradient similarity average for all folds
    avg_gradient_sim = {}
    count_keys = {}
    for sim_dict in all_fold_gradient_similarities:
        for key, value in sim_dict.items():
            avg_gradient_sim[key] = avg_gradient_sim.get(key, 0.0) + value
            count_keys[key] = count_keys.get(key, 0) + 1
    for key in avg_gradient_sim:
        avg_gradient_sim[key] /= count_keys[key]
    print("\n=== Average Final Gradient Similarity across folds ===")
    print(avg_gradient_sim)

    print("\n======================= K-Fold data splitting and training completed =======================")


if __name__ == "__main__":
    main()

Combination ('liv', 'lun', 'sto'): Number of training samples = 9, Number of validation samples = 1
Combination ('liv', 'lun', 'mgl'): Number of training samples = 10, Number of validation samples = 2
Combination ('liv', 'sto', 'mgl'): Number of training samples = 4, Number of validation samples = 1
Combination ('lun', 'sto', 'mgl'): Number of training samples = 4, Number of validation samples = 2

============================= Starting pre-training with 3-task combinations =============================

--- Epoch 1 ---

--- Epoch 2 ---

--- Epoch 3 ---

--- Epoch 4 ---

--- Epoch 5 ---

--- Epoch 6 ---

--- Epoch 7 ---

--- Epoch 8 ---

--- Epoch 9 ---

--- Epoch 10 ---

--- Epoch 11 ---

--- Epoch 12 ---

--- Epoch 13 ---

--- Epoch 14 ---

--- Epoch 15 ---

--- Epoch 16 ---

--- Epoch 17 ---

--- Epoch 18 ---

--- Epoch 19 ---

--- Epoch 20 ---

--- Epoch 21 ---

--- Epoch 22 ---

--- Epoch 23 ---

--- Epoch 24 ---

--- Epoch 25 ---

--- Epoch 26 ---

--- Epoch 27 ---

--- Epoch 28 

EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
Validation loss decreased (0.013102 --> 0.013072).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.013072 --> 0.013052).  Saving model ...
Validation loss decreased (0.013052 --> 0.012915).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.012915 --> 0.012867).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
Validation loss decreased (0.012867 --> 0.012851).  Saving model ...
Validation loss decreased (0.012851 --> 0.012832).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
Validation loss decreased (0.012832 --> 0.012788).  Saving model ...
EarlyStopping counter: 1 ou

EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
EarlyStopping counter: 8 out of 10
EarlyStopping counter: 9 out of 10
EarlyStopping counter: 10 out of 10
Early stopping at epoch 354. Best validation loss: 0.0110

=== Final Gradient Similarity Analysis for Fold 1 ===
Final Gradient similarity for fold 1 (tasks (0, 1, 2, 3)): {(0, 1): 0.6781105995178223, (0, 2): 0.494159996509552, (0, 3): -0.2593268156051636, (1, 2): 0.40788155794143677, (1, 3): -0.24979284405708313, (2, 3): 0.06747868657112122}
Checkpoint saved to /data/home/dbswn0814/2025JCM/model/multi task/model/checkpoint_full_retrained_fold_1.pth.tar with keys: dict_keys(['state_dict', 'optimizer_state_dict'])

============================= Retraining with full data completed for Fold 1 =============================

Saved final model weights for Fold 1: /data/home/dbswn0814/2025JCM/model/multi task/model/fold_1_final.pth

======================= Fold 2/20 =======================
Fold 2: Train=293, Validation=

Validation loss decreased (0.016037 --> 0.016001).  Saving model ...
Validation loss decreased (0.016001 --> 0.015969).  Saving model ...
Validation loss decreased (0.015969 --> 0.015934).  Saving model ...
Validation loss decreased (0.015934 --> 0.015914).  Saving model ...
Validation loss decreased (0.015914 --> 0.015899).  Saving model ...
Validation loss decreased (0.015899 --> 0.015876).  Saving model ...
Validation loss decreased (0.015876 --> 0.015807).  Saving model ...
Validation loss decreased (0.015807 --> 0.015778).  Saving model ...
Validation loss decreased (0.015778 --> 0.015722).  Saving model ...
Validation loss decreased (0.015722 --> 0.015689).  Saving model ...
Validation loss decreased (0.015689 --> 0.015687).  Saving model ...
Validation loss decreased (0.015687 --> 0.015649).  Saving model ...
Validation loss decreased (0.015649 --> 0.015617).  Saving model ...
Validation loss decreased (0.015617 --> 0.015574).  Saving model ...
EarlyStopping counter: 1 out of 10

Validation loss decreased (0.012750 --> 0.012741).  Saving model ...
Validation loss decreased (0.012741 --> 0.012688).  Saving model ...
Validation loss decreased (0.012688 --> 0.012670).  Saving model ...
Validation loss decreased (0.012670 --> 0.012627).  Saving model ...
Validation loss decreased (0.012627 --> 0.012582).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.012582 --> 0.012558).  Saving model ...
Validation loss decreased (0.012558 --> 0.012548).  Saving model ...
Validation loss decreased (0.012548 --> 0.012513).  Saving model ...
Validation loss decreased (0.012513 --> 0.012504).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.012504 --> 0.012487).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.012487 --> 0.012459).  Saving model ...
Validation loss decreased (0.012459 --> 0.012455).  Saving model ...
Validation loss decreased (0.012

...

EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
Validation loss decreased (0.011973 --> 0.011911).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.011911 --> 0.011860).  Saving model ...
Validation loss decreased (0.011860 --> 0.011835).  Saving model ...
Validation loss decreased (0.011835 --> 0.011821).  Saving model ...
Validation loss decreased (0.011821 --> 0.011773).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.011773 --> 0.011754).  Saving model ...
Validation loss decreased (0.011754 --> 0.011739).  Saving model ...
Validation loss decreased (0.011739 --> 0.011665).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.011665 --> 0.011643).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.011643 --> 0.011618).  Saving model ...
Validation loss decreased (0.011618 --> 0.011595).  Saving model 

Validation loss decreased (0.009205 --> 0.009161).  Saving model ...
Validation loss decreased (0.009161 --> 0.009076).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.009076 --> 0.009045).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
Validation loss decreased (0.009045 --> 0.008989).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
Validation loss decreased (0.008989 --> 0.008980).  Saving model ...
Validation loss decreased (0.008980 --> 0.008862).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.008862 --> 0.008841).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 ou

EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.016682 --> 0.016676).  Saving model ...
Validation loss decreased (0.016676 --> 0.016642).  Saving model ...
Validation loss decreased (0.016642 --> 0.016627).  Saving model ...
Validation loss decreased (0.016627 --> 0.016607).  Saving model ...
Validation loss decreased (0.016607 --> 0.016572).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.016572 --> 0.016559).  Saving model ...
Validation loss decreased (0.016559 --> 0.016514).  Saving model ...
Validation loss decreased (0.016514 --> 0.016493).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.016493 --> 0.016471).  Saving model ...
Validation loss decreased (0.016471 --> 0.016382).  Saving model ...
Validation loss decreased (0.016382 --> 0.016337).  Saving model ...
Validation loss decreased (0.016337 --> 0.016314).  Saving model ...
Validation loss decreased (0.016

EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
Validation loss decreased (0.012347 --> 0.012187).  Saving model ...
Validation loss decreased (0.012187 --> 0.012092).  Saving model ...
Validation loss decreased (0.012092 --> 0.012004).  Saving model ...
Validation loss decreased (0.012004 --> 0.011882).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
Validation loss decreased (0.011882 --> 0.011813).  Saving model ...
Validation loss decreased (0.011813 --> 0.011785).  Saving model ...
Validation loss decreased (0.011785 --> 0.011744).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.011744 --> 0.011700).  Saving model ...
Validation loss decreased (0.011700 --> 0.011645).  Saving model ...
Validation loss decreased (0.011645 --> 0.011628).  Saving model

EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
Validation loss decreased (0.008532 --> 0.008473).  Saving model ...
Validation loss decreased (0.008473 --> 0.008456).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
Validation loss decreased (0.008456 --> 0.008443).  Saving model ...
Validation loss decreased (0.008443 --> 0.008405).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.008405 --> 0.008359).  Saving model ...
EarlyStopping counter: 1 out of 10
Validation loss decreased (0.008359 --> 0.008299).  Saving model ...
EarlyStopping counter: 1 out of 10
EarlyStopping counter: 2 out of 10
EarlyStopping counter: 3 out of 10
EarlyStopping counter: 4 out of 10
EarlyStopping counter: 5 out of 10
EarlyStopping counter: 6 out of 10
EarlyStopping counter: 7 out of 10
Validation loss decreased (0.008299 --> 0.008290).  Saving model ...
Validation loss decreased (